In [2]:
#Sihao Ren // sren03 // 947105531
using Flux
using Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using Flux: @epochs
using MLJBase
using Printf
using BSON: @save # for save weights
using Plots

In [3]:
using CSV
using DataFrames

In [4]:
df = DataFrame(CSV.File("data/nba_logreg.csv"));
zer = 0
on = 0
for r in eachrow(df)
    if r.TARGET_5Yrs == 0
        zer = zer + 1
    else
        on = on + 1
    end
end
increase = on-zer

for r in eachrow(df)
    if r.TARGET_5Yrs == 0 && increase > 0
        push!(df, r)
        increase = increase - 1
    end
        
end
df[shuffle(axes(df, 1)), :]; #https://github.com/JuliaData/DataFrames.jl/issues/2048

In [7]:
typeof(df)

DataFrame

In [66]:
rawdata = readdlm("data/nba_logreg.csv")'


LoadError: UndefVarError: readdlm not defined

In [82]:
train = df[1:1100, :];
test = df[1101:1662, :];
y_train, y_test = onehotbatch(train[:, end], 0:1), onehotbatch(test[:, end], 0:1)

x_train = train[:, 1:end-1];
x_test = test[:, 1:end-1];


In [83]:
x_train

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?
1,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0
2,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5
3,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4
4,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6
5,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0
6,75,11.4,3.7,1.5,3.5,42.3,0.3,1.1,32.5
7,62,10.9,6.6,2.5,5.8,43.5,0.0,0.1,50.0
8,48,10.3,5.7,2.3,5.4,41.5,0.4,1.5,30.0
9,65,9.9,2.4,1.0,2.4,39.2,0.1,0.5,23.3


In [85]:
x_train = transpose(Matrix(x_train))
x_test = transpose(Matrix(x_test));




In [86]:
train_data = (x_train, y_train)
test_data = (x_test, y_test)

(Union{Missing, Float64}[50.0 49.0 … 26.0 38.0; 19.8 16.0 … 10.3 8.5; … ; 0.3 0.9 … 0.4 0.4; 1.4 0.9 … 0.5 0.5], Bool[0 0 … 1 1; 1 1 … 0 0])

In [87]:
model = Chain(Dense(19, 2*19, sigmoid), Dense(2*19, 2), softmax)

Chain(
  Dense(19, 38, σ),                     # 760 parameters
  Dense(38, 2),                         # 78 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 838 parameters, 3.523 KiB.

In [88]:
loss(x, y) = crossentropy(model(x), y) 

optim = ADAM()
dataset = repeated((x_train,y_train),1);
myaccuracy(x, y) = mean(onecold(model(x)) .== onecold(y))



myaccuracy (generic function with 1 method)

In [89]:
Flux.train!(loss, Flux.params(model), zip(x_train, y_train), optim)

LoadError: MethodError: no method matching (::Dense{typeof(σ), Matrix{Float32}, Vector{Float32}})(::Float64)
[0mClosest candidates are:
[0m  (::Dense)([91m::AbstractVecOrMat{T} where T[39m) at C:\Users\xkzmx\.julia\packages\Flux\qAdFM\src\layers\basic.jl:156
[0m  (::Dense)([91m::AbstractArray[39m) at C:\Users\xkzmx\.julia\packages\Flux\qAdFM\src\layers\basic.jl:161